In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:

import os
import random
import shutil
import time
import warnings
import datetime
import sys
import copy

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib.colors import ListedColormap
from pprint import pprint
import matplotlib.ticker as ticker



os.chdir('./')
# EdMIPS/models ディレクトリへのパスを追加
sys.path.append('./models')
from models.quant_efficientnet import BasicCNNBlock
import models as models

# カレントディレクトリを 'EdMIPS' に変更
os.chdir('.')



In [3]:
device_count = torch.cuda.device_count()
print('Number of devices: {}'.format(device_count))
# setting device
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'is GPU avilable? :{torch.cuda.is_available()}')

Number of devices: 2
is GPU avilable? :True


In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [5]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')
    
    acc1_avg = 0
    acc5_avg = 0
    # switch to evaluate mode
    model.eval()
    model.to(device)

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)
            
            # cifar-10のみ
            images = F.interpolate(images,size=(224,224),mode='bicubic')
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            acc1_avg += float(acc1)
            acc5_avg += float(acc5)
            # check
            # print(f'{i}  acc1:{int(acc1)}  acc5:{int(acc5)}')
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        
    acc1_avg = acc1_avg / len(val_loader)
    acc5_avg = acc5_avg / len(val_loader)
    return acc1_avg , acc5_avg
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            #correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [6]:
def load_cifar10(batch):
    train_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    val_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                 [0.5, 0.5, 0.5],  # RGB 平均
                                 [0.5, 0.5, 0.5]  # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    return train_loader,val_loader
def load_cifar100(batch):
    train_loader = DataLoader(
        datasets.CIFAR100('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    val_loader = DataLoader(
        datasets.CIFAR100('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                 [0.5, 0.5, 0.5],  # RGB 平均
                                 [0.5, 0.5, 0.5]  # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    return train_loader,val_loader
def load_imagenet100(DATASETDIR,config):
    # Data loading code
    traindir = os.path.join(DATASETDIR, 'train')
    valdir = os.path.join(DATASETDIR , 'val')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

    crop_size, short_size = 224, 256


    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(crop_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    print(f'crop size:{crop_size},short_size:{short_size}')
    train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], shuffle=(train_sampler is None),
        num_workers=config['workers'], pin_memory=True, 
        sampler=train_sampler)

    val_dataset =datasets.ImageFolder(
        valdir,
        transforms.Compose([
            transforms.Resize(short_size),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor(),
            normalize,
        ]))
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config['batch_size'], shuffle=False,
        num_workers=config['workers'], pin_memory=True)

    # print(len(train_loader))
    # print(len(val_loader))
    return train_loader,val_loader

In [7]:
def find_B(train_loader,model,splitting_points):
    model.to(device)
    model.change_learning_method(False)
    with torch.no_grad():
        for i, (images, target) in enumerate(train_loader):
            images = images.to(device)
            # cifar-10のみ
            images = F.interpolate(images,size=(224,224),mode='bicubic')
            model.eval()
            if i==0:
                features_list = torch.stack(model.feature_extractor(images,splitting_points))
            else:
                features = torch.stack(model.feature_extractor(images,splitting_points))
                features_list = torch.cat([features_list,features],dim = 1)
    # check
    print([len(v) for v in features_list])
    
    
    bound_threshold_medi = torch.median(features_list,dim = 1).values
    bound_threshold_mean = torch.mean(features_list,dim = 1)
    
    # check 
    print(f"bound threshold median : {bound_threshold_medi}")
    print(f"bound threshold mean : {bound_threshold_mean}")

    return bound_threshold_medi


In [8]:
def get_natural_bottlenecks(model, input_size, act_bits, compressive_only=True):
    # 各層のinputサイズを計算して、圧縮率が最も高い層を探す
    natural_bottlenecks = []
    best_compression = 1.0
    cnn_count = 0  # CNNレイヤーのカウント
    input_bit = 8 # 入力のbit数
    min_bit = 8  # 探索する最小のbit数←使って無くない？
    bit_compression = [act_bit / input_bit for act_bit in act_bits]

    device = next(model.parameters()).device
    
    mock_input = torch.randn(1, 3, input_size, input_size).to(device)
    previous_size = torch.prod(torch.tensor(mock_input.shape[1:])).item()

    for i, module in enumerate(model.features):
        
        block_number = i-1 # 0はfeaturesの最初のBasicCNNBlockなので、1から始める
        # print(i,block_number)
        if isinstance(module, BasicCNNBlock):
            print(module)
            print(f"Encountered BasicBlock at features.{i}")
            output = module(mock_input)
            mock_input = output.detach()
            continue
        
        input_size_layer = torch.prod(torch.tensor(mock_input.shape[1:])).item()
        # if input_size_layer * act_bits[cnn_count] < input_size * input_size * 3 * input_bit:
        if True:
            compression = float(input_size_layer) / (input_size * input_size * 3)
            compression *= bit_compression[cnn_count]
            # print(i,block_number,compression)
            if not compressive_only or compression < best_compression:
                print(i,block_number)
                natural_bottlenecks.append({
                    'layer_name': "blocks_{}".format(block_number),
                    'compression': compression,
                    'cnn_layer_number': cnn_count,  # ここでCNNレイヤーの番号を記録
                    'block_number': block_number,  
                })
                best_compression = compression
        output = module(mock_input)
        mock_input = output.detach()
        
        cnn_count += count_conv2d_layers(module)
    return natural_bottlenecks

def count_conv2d_layers(model):
    count = 0
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            count += 1
        elif isinstance(module, nn.Sequential):
            # Sequentialブロック内でさらにConv2dを探す
            for sub_module in module:
                if isinstance(sub_module, nn.Conv2d):
                    count += 1
    return count

In [9]:
config = {
    'model_name': 'quanteffnet_w8a8_with_DP',
    # 量子制度決定済み・何かしら学習したあとのアーキテクチャ
    'arch_path' : './privacy_aware_dsc/efficient_b0_w8a8/clean_ep50_cifar10/clean_ep50_cifar10.pth.tar',
    'image_size': 224,
    'batch_size': 50,
    'workers'   : 16,
    'save_dir':'./privacy_aware_dsc/efficient_b0_w8a8_imagenet100_with_DP/noise_5_50_cifar10',
    'save_name':'noise_5_50_cifar10'

}
DATASETDIR ='~/datasets/imagenet-100'
# 量子精度は決定したけど学習してないアーキテクチャ
mixefnet_dir='./arch_output/mixeffnet_b0_w2468a2468_100_csd0.01_forlossynet/arch_model_best.pth.tar'

In [10]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

In [11]:
# data loading
train_loader,val_loader = load_cifar10(batch=config['batch_size'])
print(len(train_loader),len(val_loader))
print(config['batch_size'])

# model loading
checkpoint = torch.load(config['arch_path'])
start_epoch = checkpoint['epoch']
print(start_epoch)
# best_acc1 = checkpoint['best_acc1']
# best_acc1.to(device)
# print(best_acc1)
cudnn.benchmark = True

Files already downloaded and verified
Files already downloaded and verified
1000 200
50
50


In [12]:
# baseline ノイズを加えないときの精度
# model = models.__dict__[config['model_name']](mixefnet_dir)
# U8
model = models.__dict__[config['model_name']]("")
criterion = nn.CrossEntropyLoss().to(device)
model.load_state_dict(checkpoint['state_dict'])
model.to(device)

model.change_learning_method(noisy_flag = False)
# baseline_acc1,baseline_acc5=validate(val_loader,model,criterion)
# print(f'baseline_acc1 : {baseline_acc1}, baseline_acc5 : {baseline_acc5}')

archas: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
archws: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]


In [13]:
print(model)

EfficientNet_with_DP(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (features): Sequential(
    (0): BasicCNNBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): QuantActivConv2d(
            (activ): HWGQ()
            (conv): QuantConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          )
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): QuantActivConv2d(
              (activ): HWGQ()
              (conv): QuantConv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
           

In [14]:
natural_bottlenecks = get_natural_bottlenecks(model,224,model.archas)
print(natural_bottlenecks)


BasicCNNBlock(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (silu): SiLU()
)
Encountered BasicBlock at features.0
3 2
5 4
7 6
13 12
[{'layer_name': 'blocks_2', 'compression': 0.5, 'cnn_layer_number': 9, 'block_number': 2}, {'layer_name': 'blocks_4', 'compression': 0.20833333333333334, 'cnn_layer_number': 19, 'block_number': 4}, {'layer_name': 'blocks_6', 'compression': 0.10416666666666667, 'cnn_layer_number': 29, 'block_number': 6}, {'layer_name': 'blocks_12', 'compression': 0.0625, 'cnn_layer_number': 59, 'block_number': 12}]


In [15]:
natural_bottlenecks = get_natural_bottlenecks(model,224,model.archas)
split_points = [bottleneck['block_number'] for bottleneck in natural_bottlenecks]
print(f'split points(block) : {split_points}')
B_list = find_B(train_loader,model,split_points)
print(f'Bound threshold median list : {B_list}')

BasicCNNBlock(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (silu): SiLU()
)
Encountered BasicBlock at features.0
3 2
5 4
7 6
13 12
split points(block) : [2, 4, 6, 12]


KeyboardInterrupt: 

In [ ]:
# for name, module in model.features.named_children():
#     print(name)
    
# for name, module in model.named_children():
#     print(name)

In [ ]:
# setting
natural_bottlenecks = get_natural_bottlenecks(model,config['image_size'],model.archas)
splitting_points = [bottleneck['block_number'] for bottleneck in natural_bottlenecks]
noise_scales = [round(num,1) for num in range(1,40)]
print(splitting_points)

B_list = find_B(train_loader,model,splitting_points)

Encountered BasicBlock at features.0
[2, 4, 6, 12]
[50000, 50000, 50000, 50000]
bound threshold median : tensor([6.9984, 6.9221, 6.6766, 4.1041], device='cuda:0')
bound threshold mean : tensor([6.9396, 7.0650, 7.0362, 4.3055], device='cuda:0')


In [ ]:
# check
print(B_list)

tensor([6.9984, 6.9221, 6.6766, 4.1041], device='cuda:0')


In [ ]:
# validation　ノイズを加えた時の精度測定
acc1_progress = {}
acc5_progress = {}
for splitting_point in splitting_points:
    print(f'Splitting_Point: {splitting_point}')
    model.set_splitting_point(splitting_point)
    model.set_B(B_list[splitting_points.index(splitting_point)])
    acc1_progress[splitting_point] = []
    acc5_progress[splitting_point] = []
    for noise_scale in noise_scales:
        # print(f'Noise_Scale: {noise_scale}')
        model.set_noise_scale(noise_scale)
        model.change_learning_method(noisy_flag = True)

        acc1,acc5 = validate(val_loader,model,criterion)
        acc1_progress[splitting_point].append(acc1)
        acc5_progress[splitting_point].append(acc5)
    print("done")

Splitting_Point: 2
done
Splitting_Point: 4
done
Splitting_Point: 6
done
Splitting_Point: 12
done


In [ ]:
print(acc1_progress)    
print(acc5_progress)
print("Experiment done")
print(datetime.datetime.now().strftime('%Y-%m%d_%H:%M:%S'))

{2: [10.42, 12.76, 19.97, 39.2, 56.34, 67.73, 75.17, 79.39, 82.9, 84.68, 85.9, 86.54, 87.44, 87.51, 87.93, 88.32, 88.1, 88.24, 88.16, 88.36, 88.35, 88.77, 88.63, 88.63, 88.42, 88.8, 88.55, 88.49, 88.76, 88.64, 88.78, 88.62, 88.74, 88.96, 88.62, 88.83, 88.8, 88.75, 88.73], 4: [9.81, 10.91, 16.87, 31.24, 43.73, 53.61, 61.66, 68.37, 72.94, 76.81, 79.04, 81.07, 82.54, 83.69, 84.57, 85.76, 85.76, 86.64, 86.33, 86.99, 87.36, 87.78, 87.64, 87.81, 88.13, 88.12, 88.02, 88.12, 88.32, 88.26, 88.34, 88.54, 88.53, 88.32, 88.68, 88.47, 88.51, 88.7, 88.5], 6: [10.09, 9.92, 12.14, 16.34, 25.9, 35.49, 44.56, 55.04, 61.77, 67.14, 72.32, 76.64, 78.73, 81.14, 82.21, 83.95, 84.34, 84.86, 85.84, 86.14, 86.62, 86.56, 87.09, 87.28, 87.13, 87.32, 87.83, 87.68, 87.6, 87.87, 88.11, 88.2, 88.1, 88.26, 88.18, 88.29, 88.52, 88.44, 88.47], 12: [9.5, 9.96, 10.3, 11.67, 23.12, 39.35, 51.73, 60.45, 67.53, 71.84, 75.24, 76.9, 78.72, 81.38, 82.32, 83.77, 84.54, 85.15, 85.38, 86.04, 86.52, 86.77, 87.06, 87.07, 87.46, 87.3

In [ ]:
plt.figure(figsize=[16,6])
for splitting_point in splitting_points:
    plt.plot(noise_scales,acc1_progress[splitting_point])
plt.axhline(baseline_acc1,color='midnightblue')

plt.ylim(top=90)

plt.legend(splitting_points,title="splitting point")
plt.xticks(noise_scales)
plt.xlabel("Noise_Scale")
plt.ylabel("Top1-Accuracy")
plt.title("Top1-Accuracy EfficientNet added Laplace Noise at each split point")
plt.grid()

plt.show()
plt.savefig(f"{config['save_dir']}/acc1_{config['save_name']}_v2.png",bbox_inches='tight')



<IPython.core.display.Javascript object>

In [ ]:
plt.figure(figsize=[16,6])
for splitting_point in splitting_points:
    plt.plot(noise_scales,acc5_progress[splitting_point])
plt.axhline(baseline_acc5,color='midnightblue')
plt.ylim(top=100)
plt.legend(splitting_points,title="splitting point")
plt.xticks(noise_scales)
plt.xlabel("Noise_Scale")
plt.ylabel("Top5-Accuracy")
plt.title("Top5-Accuracy EfficientNet added Laplace Noise at each split point")
plt.grid()
plt.show()
plt.savefig(f"{config['save_dir']}/acc5_{config['save_name']}_v2.png",bbox_inches='tight')

<IPython.core.display.Javascript object>